## Configuration

In [ ]:
import os
from dotenv import load_dotenv
import requests

load_dotenv()

aca_path="https://api.uat.platform.pagopa.it/aca/debt-positions-service/v1"

subscription_key = os.getenv('MY_SUBSCRIPTION_KEY')
organization = os.getenv('PAGOPA_TEST_ORGANIZATION')

segregation_code = os.getenv('PAGOPA_TEST_SEGREGATION_CODE')

print("Configuration set successfully.")

## Add dynamic values to json template

In [ ]:
import json
import random
import string

file_path = 'PaymentPosition.json'
try:
    with open(file_path, 'r') as file:
        data = json.load(file)
        
    generated_iupd = ''.join(random.choices(string.ascii_uppercase + string.digits, k=15))
    data['iupd'] = generated_iupd    
    paymentOptions = data['paymentOption']
    for paymentOption in paymentOptions:
        generated_iuv = segregation_code + ''.join(random.choices(string.digits, k=15))
        paymentOption['iuv'] = generated_iuv
        paymentOption['description'] = 'Posizione debitoria ACA test'

    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

    print(f"JSON file '{file_path}' modified successfully.")

except FileNotFoundError:
    print(f"Error: file '{file_path}' was not found.")
except json.JSONDecodeError:
    print(f"Error: file '{file_path}' is not a valid JSON.")

# CREATE

| Field | Value |
| :--- | :---: |
| Operation | Create ACA debt position |
| Body | Debt Position |
| URI | `{base-path}/organizations/{organization}/debtpositions` |

In [ ]:
url = aca_path+"/organizations/"+organization+"/debtpositions"

try:
    with open(file_path, 'r') as file:
        payload = json.load(file)
        headers = {
            'Ocp-Apim-Subscription-Key': subscription_key
        }
        response = requests.request("POST", url, headers=headers, json=payload)

        print(f"Response status code: {str(response.status_code)}")
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except json.JSONDecodeError:
    print(f"Error: Invalid JSON format in {file_path}")


## Check debt position

In [ ]:
url = aca_path+"/organizations/"+organization+"/debtpositions/" + generated_iupd

response = requests.request("GET", url, headers=headers)

print(response.text)

# UPDATE

## Modify dinamically the payment option description field inside payment position file

In [ ]:
import json

file_path = 'PaymentPosition.json'
try:
    with open(file_path, 'r') as file:
        data = json.load(file)

    paymentOptions = data['paymentOption']
    for paymentOption in paymentOptions:
        paymentOption['description'] = 'Posizione debitoria ACA test updated'

    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

    print(f"JSON file '{file_path}' modified successfully.")

except FileNotFoundError:
    print(f"Error: file '{file_path}' was not found.")
except json.JSONDecodeError:
    print(f"Error: file '{file_path}' is not a valid JSON.")

## Update operation

| Field | Value |
| :--- | :---: |
| Operation | Update ACA debt position |
| Body | Debt Position |
| URI | `{base-path}/organizations/{organization}/debtpositions/{iupd}` |

In [ ]:
url = aca_path+"/organizations/"+organization+"/debtpositions/"+generated_iupd

try:
    with open(file_path, 'r') as file:
        payload = json.load(file)
        headers = {
            'Ocp-Apim-Subscription-Key': subscription_key
        }
        response = requests.request("PUT", url, headers=headers, json=payload)

        print(f"Response status code: {str(response.status_code)}")
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except json.JSONDecodeError:
    print(f"Error: Invalid JSON format in {file_path}")

## Check debt position

In [ ]:
url =aca_path+"/organizations/"+organization+"/debtpositions/" + generated_iupd

response = requests.request("GET", url, headers=headers)

print(response.text)

# PUBLISH

## Publish operation

| Field | Value |
| :--- | :---: |
| Operation | Publish |
| URI | `{base-path}/organizations/{organization}/debtpositions/{iupd}/publish` |

In [ ]:
url =f"{aca_path}/organizations/{organization}/debtpositions/{generated_iupd}/publish"

response = requests.request("POST", url, headers=headers)

print(response.text)